In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Loading Data

# data = pd.read_csv('breast-cancer.data')
# data = pd.read_csv('adult.data')
# # 
# data = data.replace({'?': 0})
# data = data.dropna(axis=0) 
# print(data.head())


# print(df.columns)
# df = pd.read_csv("obesity.csv", on_bad_lines='skip') # Please Uncomment for external dataset 
# df=df.drop(df.columns[1], axis=1)

In [ ]:
class DSTreeNode: 
    def __init__(self):
        self.left_child  = None
        self.right_child = None
        self.feature     = None
        self.threashold  = None 
        
        self.leafv       = None
        
    def set_params(self,threashold, feature):
        self.threashold = threashold
        self.feature = feature    
        
    def set_children(self,left_child,right_child):
        self.left_child  = left_child
        self.right_child = right_child

    def get_params(self):
        return (self.threashold,self.feature) 
         

In [ ]:
class DecisionTreeClassifier: 
    def __init__(self, max_depth, min_samples_split=2,loss='GINI'): 
        self.tree              = None
        self.loss = loss   
        self.max_depth         = max_depth
        self.min_samples_split = min_samples_split 



    def predict_model(self,row, node=None): 
        result = []
        for r in range(row.shape[0]):
            result.append(self.travel_tree(self.tree,row[r,:])) 
        return(np.array(result).flatten())    

    def travel_tree(self,node,rowdata): 
      if node.leafv is not None: 
          return(node.leafv)
      else: 
          
          (s,f) = node.get_params() 
          if (rowdata[f] <= s):
              return(self.travel_tree(node.left_child,rowdata))
          else:
              return(self.travel_tree(node.right_child,rowdata))

    def calculate_node_impurity(self,table):  
        if self.loss == 'Gain_Ratio':
            imp = self.calculate_gain_ratio(table) 
        elif self.loss == 'Information_Gain':
            imp = self.calculate_IG(table) 
        elif self.loss == 'GINI':
            imp = self.calculate_gini_index(table) 
        return(imp)

    def calculate_gini_index(self,table): 
        Gini = 0 
        for c in np.unique(table[:,-1]): 
            p = table[table[:,-1]==c].shape[0]/table.shape[0] 
            Gini += p*(1-p) 
        return(Gini)
     
    def calculate_IG(self,table): 
        Entropy = 0 
        for c in np.unique(table[:,-1]): 
            p = table[table[:,-1]==c].shape[0]/table.shape[0] 
            Entropy -= p*np.log2(p) 
        return(Entropy)
     
    def calculate_gain_ratio(self,table): 
        GR = 0 
        for c in np.unique(table[:,-1]): 
            p = table[table[:,-1]==c].shape[0]/table.shape[0] 
            
            GR += p*np.log2(p) 
        return(GR)

    
    
    def build_tree(self,node, table,level):     

        table_right = table_left = split = ip_node = feature = None 

        msamp = (self.min_samples_split <= table.shape[0])
        depth = (self.max_depth is None) or (self.max_depth >= (level+1)) #left node check 
        n_cls = np.unique(table[:,-1]).shape[0] != 1
         
        if not (depth and msamp and n_cls): 
            node.leafv = self.check_leaf_value(table)
            return
         
        else:  
            for feat in range(table.shape[1]-1):
                for splt in np.unique(table[:,feat]): 
                    table_low, table_high = table[table[:,feat]<=splt], table[table[:,feat]>splt]  # We are splitting to left and right 
                    if table_low.size and table_high.size: 
                        dl = (table_low.shape[0]/table.shape[0])*self.calculate_node_impurity(table_low)
                        dh = (table_high.shape[0]/table.shape[0])*self.calculate_node_impurity(table_high) 
                        ip  =  dl + dh
                        if (ip_node is None) or (ip < ip_node):
                            ip_node, feature, split = ip, feat, splt 
                            table_left, table_right  = table_low, table_high 

            node.set_params(split,feature)  
            node.set_children(DSTreeNode(),DSTreeNode()) 

            self.build_tree(node.right_child,table_right,level+1)
            self.build_tree(node.left_child,table_left,level+1)
             
    
 
    def train_model(self,row,c, n=""): 
        table = np.concatenate((row,c.reshape(-1,1)),axis=1) 
        self.tree = DSTreeNode() 
        self.build_tree(self.tree,table,1)
         
    
 
    def check_leaf_value(self,D):
         return(stats.mode(D[:,-1])[0])

In [ ]:
data = load_breast_cancer()

start_time = time.time()
X    = data.data
y    = data.target
nsampclass = 100

y0, y1, X0, X1 = y[y==0], y[y==1], X[y==0], X[y==1] 

idx0 = np.random.choice([i for i in range(y0.shape[0])],size=nsampclass,replace=False)
idx1 = np.random.choice([i for i in range(y1.shape[0])],size=nsampclass,replace=False) 

y_train0, y_train1, X_train0, X_train1 = y0[idx0], y1[idx1], X0[idx0,:], X1[idx1,:] 
y_train  = np.concatenate((y_train0,y_train1))
X_train  = np.concatenate((X_train0,X_train1))


y_test0, y_test1, X_test0, X_test1, y_test   = np.delete(y0,idx0), np.delete(y1,idx1), np.delete(X0,idx0,axis=0), np.delete(X1,idx1,axis=0), np.concatenate((y_test0,y_test1)) 
X_test  = np.concatenate((X_test0,X_test1))

 
clf = DecisionTreeClassifier(max_depth=5, loss = 'Information_Gain')
clf.train_model(X_train,y_train) 
yp = clf.predict_model(X_test)
end_time = time.time()
print("Time Taken %s seconds" % (round(end_time - start_time, 4)))



Time Taken 1.1469 seconds


In [ ]:
print("Data Accuracy: %.2f" % accuracy_score(y_test,yp))
print("Data Sensitivity: %.2f" % precision_score(y_test,yp))
print("Data Specificity: %.2f" % recall_score(y_test,yp))
 

Data Accuracy: 0.90
Data Sensitivity: 0.97
Data Specificity: 0.88


In [ ]:
process = psutil.Process(os.getpid())
print("Total Used Memory for FP Growth Mining: ", end='')
print(process.memory_info().rss)

Total Used Memory for FP Growth Mining: 199585792
